In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
import os


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

2023-05-17 23:15:30.347084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 23:15:30.479390: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-17 23:15:30.833461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/themandalorian/anaconda3/envs/ml-on-mc/lib/
2023-05-17 23:15:30.833516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [3]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.1))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_reduced_5Conv.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=400, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 8, 8, 32)          0

2023-05-17 23:15:38.348656: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 23:15:38.348857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 23:15:38.348981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 23:15:38.349257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 23:15:38.349389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Epoch 1/400


2023-05-17 23:15:40.115845: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


  1/782 [..............................] - ETA: 25:14 - loss: 6.3581 - accuracy: 0.0469

...

782/782 [==============================] - 3s 4ms/step - loss: 0.8100 - accuracy: 0.7201 - val_loss: 0.7231 - val_accuracy: 0.7465
Epoch 172/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8037 - accuracy: 0.7222 - val_loss: 0.7270 - val_accuracy: 0.7463
Epoch 173/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8079 - accuracy: 0.7194 - val_loss: 0.7222 - val_accuracy: 0.7498
Epoch 174/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8078 - accuracy: 0.7214 - val_loss: 0.7047 - val_accuracy: 0.7536
Epoch 175/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8050 - accuracy: 0.7228 - val_loss: 0.7254 - val_accuracy: 0.7479
Epoch 176/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8038 - accuracy: 0.7213 - val_loss: 0.7057 - val_accuracy: 0.7524
Epoch 177/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8048 - accuracy: 0.7222 - val_loss: 0.6914 - val_accuracy: 0.7617

782/782 [==============================] - 3s 4ms/step - loss: 0.7536 - accuracy: 0.7424 - val_loss: 0.6564 - val_accuracy: 0.7731
Epoch 228/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7540 - accuracy: 0.7411 - val_loss: 0.6498 - val_accuracy: 0.7731
Epoch 229/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7522 - accuracy: 0.7423 - val_loss: 0.6559 - val_accuracy: 0.7754
Epoch 230/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7448 - accuracy: 0.7437 - val_loss: 0.6619 - val_accuracy: 0.7715
Epoch 231/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7498 - accuracy: 0.7418 - val_loss: 0.6873 - val_accuracy: 0.7599
Epoch 232/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7466 - accuracy: 0.7434 - val_loss: 0.6594 - val_accuracy: 0.7703
Epoch 233/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7425 - accuracy: 0.7445 - val_loss: 0.6576 - val_accuracy: 0.7721

782/782 [==============================] - 3s 4ms/step - loss: 0.7132 - accuracy: 0.7555 - val_loss: 0.5979 - val_accuracy: 0.7925
Epoch 284/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7108 - accuracy: 0.7567 - val_loss: 0.6210 - val_accuracy: 0.7864
Epoch 285/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7108 - accuracy: 0.7576 - val_loss: 0.6294 - val_accuracy: 0.7848
Epoch 286/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7066 - accuracy: 0.7574 - val_loss: 0.6083 - val_accuracy: 0.7896
Epoch 287/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7180 - accuracy: 0.7538 - val_loss: 0.6104 - val_accuracy: 0.7873
Epoch 288/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7031 - accuracy: 0.7584 - val_loss: 0.6076 - val_accuracy: 0.7916
Epoch 289/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7049 - accuracy: 0.7619 - val_loss: 0.6426 - val_accuracy: 0.7782

782/782 [==============================] - 3s 4ms/step - loss: 0.6776 - accuracy: 0.7677 - val_loss: 0.5898 - val_accuracy: 0.7974
Epoch 340/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6869 - accuracy: 0.7638 - val_loss: 0.6365 - val_accuracy: 0.7798
Epoch 341/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6826 - accuracy: 0.7663 - val_loss: 0.5954 - val_accuracy: 0.7953
Epoch 342/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6835 - accuracy: 0.7655 - val_loss: 0.5972 - val_accuracy: 0.7971
Epoch 343/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6810 - accuracy: 0.7669 - val_loss: 0.5856 - val_accuracy: 0.7975
Epoch 344/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6834 - accuracy: 0.7663 - val_loss: 0.6122 - val_accuracy: 0.7890
Epoch 345/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6760 - accuracy: 0.7658 - val_loss: 0.6104 - val_accuracy: 0.7920

782/782 [==============================] - 3s 4ms/step - loss: 0.6673 - accuracy: 0.7728 - val_loss: 0.6087 - val_accuracy: 0.7877
Epoch 396/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6614 - accuracy: 0.7728 - val_loss: 0.6219 - val_accuracy: 0.7823
Epoch 397/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6626 - accuracy: 0.7716 - val_loss: 0.5840 - val_accuracy: 0.7962
Epoch 398/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6585 - accuracy: 0.7732 - val_loss: 0.5915 - val_accuracy: 0.7962
Epoch 399/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6624 - accuracy: 0.7743 - val_loss: 0.6059 - val_accuracy: 0.7891
Epoch 400/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6637 - accuracy: 0.7719 - val_loss: 0.5868 - val_accuracy: 0.7970


In [4]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI_reduced_5Conv-PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of reduced base quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmpj7o0uyn6/assets


INFO:tensorflow:Assets written to: /tmp/tmpj7o0uyn6/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 23:36:39.258717: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 23:36:39.258746: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 23:36:39.259574: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpj7o0uyn6
2023-05-17 23:36:39.262882: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 23:36:39.262900: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpj7o0uyn6
2023-05-17 23:36:39.270554: I tensorflow/

== Input details ==
name: serving_default_conv2d_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of reduced base quantized to int8 model is 79.65%


In [6]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

c_model_name = 'cifar10_base_reduced_5conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [ ]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.1))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

# compile model
opt = SGD(lr=0.001, momentum=0.9)
quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_reduced_5Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=400,validation_data=(x_test, y_test),
                     callbacks=[checkpoint_callback])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 8, 8, 32)         

/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/400
782/782 [==============================] - 9s 10ms/step - loss: 2.3290 - accuracy: 0.1160 - val_loss: 2.2739 - val_accuracy: 0.1812
Epoch 2/400
782/782 [==============================] - 8s 10ms/step - loss: 2.1992 - accuracy: 0.1764 - val_loss: 2.1951 - val_accuracy: 0.1759
Epoch 3/400
782/782 [==============================] - 7s 9ms/step - loss: 2.0843 - accuracy: 0.2071 - val_loss: 2.0310 - val_accuracy: 0.2222
Epoch 4/400
782/782 [==============================] - 8s 10ms/step - loss: 1.9851 - accuracy: 0.2327 - val_loss: 1.9191 - val_accuracy: 0.2784
Epoch 5/400
782/782 [==============================] - 8s 10ms/step - loss: 1.9152 - accuracy: 0.2573 - val_loss: 1.8595 - val_accuracy: 0.3164
Epoch 6/400
782/782 [==============================] - 7s 10ms/step - loss: 1.8630 - accuracy: 0.2776 - val_loss: 1.7667 - val_accuracy: 0.3490
Epoch 7/400
782/782 [==============================] - 8s 10ms/step - loss: 1.8199 - accuracy: 0.2980 - val_loss: 1.8332 - val_accuracy: 

782/782 [==============================] - 7s 9ms/step - loss: 1.1263 - accuracy: 0.5954 - val_loss: 1.0772 - val_accuracy: 0.6249
Epoch 58/400
782/782 [==============================] - 7s 10ms/step - loss: 1.1114 - accuracy: 0.6006 - val_loss: 1.0222 - val_accuracy: 0.6361
Epoch 59/400
782/782 [==============================] - 8s 10ms/step - loss: 1.1065 - accuracy: 0.6006 - val_loss: 1.0207 - val_accuracy: 0.6355
Epoch 60/400
782/782 [==============================] - 7s 10ms/step - loss: 1.1101 - accuracy: 0.6032 - val_loss: 0.9829 - val_accuracy: 0.6526
Epoch 61/400
782/782 [==============================] - 7s 9ms/step - loss: 1.0982 - accuracy: 0.6081 - val_loss: 0.9567 - val_accuracy: 0.6605
Epoch 62/400
782/782 [==============================] - 7s 10ms/step - loss: 1.0990 - accuracy: 0.6075 - val_loss: 1.0581 - val_accuracy: 0.6292
Epoch 63/400
782/782 [==============================] - 8s 10ms/step - loss: 1.0926 - accuracy: 0.6127 - val_loss: 1.0007 - val_accuracy: 0.6447


782/782 [==============================] - 7s 10ms/step - loss: 0.9209 - accuracy: 0.6770 - val_loss: 0.8228 - val_accuracy: 0.7054
Epoch 114/400
782/782 [==============================] - 8s 10ms/step - loss: 0.9164 - accuracy: 0.6793 - val_loss: 0.7959 - val_accuracy: 0.7176
Epoch 115/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9213 - accuracy: 0.6787 - val_loss: 0.8047 - val_accuracy: 0.7188
Epoch 116/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9058 - accuracy: 0.6849 - val_loss: 0.8204 - val_accuracy: 0.7082
Epoch 117/400
782/782 [==============================] - 7s 10ms/step - loss: 0.9020 - accuracy: 0.6835 - val_loss: 0.8390 - val_accuracy: 0.7038
Epoch 118/400
782/782 [==============================] - 7s 10ms/step - loss: 0.9069 - accuracy: 0.6817 - val_loss: 0.8214 - val_accuracy: 0.7107
Epoch 119/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9052 - accuracy: 0.6831 - val_loss: 0.7636 - val_accuracy: 0.

In [ ]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_gpu_Base-CubeAI_reduced_5Conv_QAT.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_gpu_Base-CubeAI_reduced_5Conv_QAT.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

In [ ]:
c_model_name = 'cifar10_base_reduced_5conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))